In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
# Veri yükleme (örnek bir dataset yüklenmelidir)
df = pd.read_csv("/content/sample_data/csv_building_structure.csv")

In [5]:
# Özellikler ve hedef değişkeni ayırma
features = [
    "plinth_area_sq_ft",
    "land_surface_condition",
    "count_floors_pre_eq",
    "height_ft_pre_eq",
    "roof_type",
    "age_building",
    "foundation_type",
    "ground_floor_type",
    "other_floor_type",
    "has_superstructure_adobe_mud",
    "has_superstructure_stone_flag",
    "has_superstructure_cement_mortar_stone",
    "has_superstructure_mud_mortar_brick",
    "has_superstructure_cement_mortar_brick",
    "has_superstructure_timber",
    "has_superstructure_bamboo",
    "has_superstructure_rc_non_engineered",
    "has_superstructure_rc_engineered",
    "has_superstructure_other",
]

X = df[features]
y = df['damage_grade']  # Hedef değişken

In [7]:
def extract_grade_number(grade):
  if pd.isnull(grade):
    return np.nan
  else:
    return int(grade.split()[-1])

df['damage_grade'] = df['damage_grade'].apply(extract_grade_number)

In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Kategorik sütunları bulma
categorical_columns = X.select_dtypes(include=['object']).columns

# Kategorik sütunları Label Encoding ile sayısallaştırma
for col in categorical_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Özellikleri standardize etme
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Geri kalan kod aynı


<ipython-input-9-f84cae3e4161>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])
<ipython-input-9-f84cae3e4161>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])
<ipython-input-9-f84cae3e4161>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [10]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Özellikleri standardize etme
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# Veriyi eğitim ve test setlerine bölme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 1. Random Forest Modeli
rf_model = RandomForestClassifier(
    n_estimators=750,
    max_features=25,
    min_samples_leaf=5,
    random_state=42
)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)

In [ ]:
### 2. Neural Network Modeli
nn_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')  # Sınıf sayısı
])

nn_model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

In [ ]:
# Modeli eğitme
nn_history = nn_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Test seti üzerinde tahminler
nn_pred = np.argmax(nn_model.predict(X_test), axis=1)
nn_accuracy = accuracy_score(y_test, nn_pred)

In [ ]:
### 3. Sonuçları Karşılaştırma ve Grafik
models = ['Random Forest', 'Neural Network']
accuracies = [rf_accuracy, nn_accuracy]

plt.bar(models, accuracies, color=['blue', 'orange'])
plt.ylim(0, 1)
plt.title('Model Accuracy Comparison')
plt.ylabel('Accuracy')
plt.show()

# Her iki modelin doğruluğunu yazdırma
print(f"Random Forest Doğruluk: {rf_accuracy:.2f}")
print(f"Neural Network Doğruluk: {nn_accuracy:.2f}")